# Introduction to Vector

Vector is a Python library for 2D, 3D, and [Lorentz vectors](https://en.wikipedia.org/wiki/Special_relativity#Physics_in_spacetime), especially _arrays of vectors_, to solve common physics problems in a NumPy-like way.

Main features of Vector:

   * Pure Python with NumPy as its only dependency. This makes it easier to install.
   * Vectors may be represented in a variety of coordinate systems: Cartesian, cylindrical, spherical, and any combination of these with time or proper time for Lorentz vectors. In all, there are 12 coordinate systems: {$x$-$y$ vs $\rho$-$\phi$ in the azimuthal plane} × {$z$ vs $\theta$ vs $\eta$ longitudinally} × {$t$ vs $\tau$ temporally}.
   * Uses names and conventions set by [ROOT](https://root.cern/)'s [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html) and [Math::LorentzVector](https://root.cern.ch/doc/master/classROOT_1_1Math_1_1LorentzVector.html), as well as [scikit-hep/math](https://github.com/scikit-hep/scikit-hep/tree/master/skhep/math), [uproot-methods TLorentzVector](https://github.com/scikit-hep/uproot3-methods/blob/master/uproot3_methods/classes/TLorentzVector.py), [henryiii/hepvector](https://github.com/henryiii/hepvector), and [coffea.nanoevents.methods.vector](https://coffeateam.github.io/coffea/modules/coffea.nanoevents.methods.vector.html).
   * Implemented on a variety of backends:
      - pure Python objects
      - NumPy arrays of vectors (as a [structured array](https://numpy.org/doc/stable/user/basics.rec.html) subclass)
      - [Awkward Arrays](https://awkward-array.org/) of vectors
      - potential for more: CuPy, TensorFlow, Torch, JAX...
   * Implemented in [Numba](https://numba.pydata.org/) for JIT-compiled calculations on vectors.
   * Distinction between geometrical vectors, which have a minimum of attribute and method names, and vectors representing momentum, which have synonyms like `pt` = `rho`, `energy` = `t`, `mass` = `tau`.

This notebook requires Vector, NumPy, Awkward Array, and Numba to run.

In [1]:
import vector
import numpy as np
import awkward as ak
import numba as nb

## Constructing a vector or an array of vectors

The easiest way to create one or many vectors is with

   * `vector.obj` to make a pure Python vector object,
   * `vector.array` to make a NumPy array of vectors (this is a subclass of `np.ndarray`),
   * `vector.Array` to make an Awkward Array of vectors.

### Pure Python vectors

In [2]:
vector.obj(x=3, y=4)   # Cartesian 2D vector

vector.obj(x=3, y=4)

In [3]:
vector.obj(rho=5, phi=0.9273)   # same in polar coordinates

vector.obj(rho=5, phi=0.9273)

In [4]:
vector.obj(x=3, y=4).isclose(vector.obj(rho=5, phi=0.9273))   # use "isclose" unless they are exactly equal

True

In [5]:
vector.obj(x=3, y=4, z=-2)   # Cartesian 3D vector

vector.obj(x=3, y=4, z=-2)

In [6]:
vector.obj(x=3, y=4, z=-2, t=10)   # Cartesian 4D vector

vector.obj(x=3, y=4, z=-2, t=10)

In [7]:
vector.obj(rho=5, phi=0.9273, eta=-0.39, t=10)   # in rho-phi-eta-t cylindrical coordinates

vector.obj(rho=5, phi=0.9273, eta=-0.39, t=10)

In [8]:
vector.obj(pt=5, phi=0.9273, eta=-0.39, E=10)   # use momentum-synonyms to get a momentum vector

vector.obj(pt=5, phi=0.9273, eta=-0.39, E=10)

In [9]:
vector.obj(rho=5, phi=0.9273, eta=-0.39, t=10) == vector.obj(pt=5, phi=0.9273, eta=-0.390035, E=10)

False

In [10]:
vector.obj(rho=5, phi=0.9273, eta=-0.39, t=10).tau   # geometrical vectors have to use geometrical names ("tau", not "mass")

8.426194916448265

In [11]:
vector.obj(pt=5, phi=0.9273, eta=-0.39, E=10).mass   # momentum vectors can use momentum names (as well as geometrical ones)

8.426194916448265

In [12]:
vector.obj(pt=5, phi=0.9273, theta=1.9513, mass=8.4262)   # any combination of azimuthal, longitudinal, and temporal coordinates is allowed

vector.obj(pt=5, phi=0.9273, theta=1.9513, mass=8.4262)

In [13]:
vector.obj(x=3, y=4, z=-2, t=10).isclose(vector.obj(pt=5, phi=0.9273, theta=1.9513, mass=8.4262))

True

In [14]:
# Test instance type for any level of granularity.
(
    isinstance(vector.obj(x=1.1, y=2.2), vector.Vector),                                    # is a vector or array of vectors
    isinstance(vector.obj(x=1.1, y=2.2), vector.Vector2D),                                  # is 2D (not 3D or 4D)
    isinstance(vector.obj(x=1.1, y=2.2), vector.VectorObject2D),                            # 
    isinstance(vector.obj(px=1.1, py=2.2), vector.Momentum),                                # has momentum synonyms
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4), vector.Planar),                      # has transverse plane (2D, 3D, or 4D)
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4), vector.Spatial),                     # has all spatial coordinates (3D or 4D)
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4), vector.Lorentz),                     # has temporal coordinates (4D)
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4).azimuthal, vector.AzimuthalXY),       # azimuthal coordinate type
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4).longitudinal, vector.LongitudinalZ),  # longitudinal coordinate type
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4).temporal, vector.TemporalT),          # temporal coordinate type
)

(True, True, True, True, True, True, True, True, True, True)

### NumPy arrays of vectors

In [15]:
vector.array

<function vector.backends.numpy_.array(*args: Any, **kwargs: Any) -> vector.backends.numpy_.VectorNumpy>